In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import os

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'cortex'

# Get keys from the environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if langchain_api_key:
    os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
else:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment.")

if groq_api_key:
    os.environ['GROQ_API_KEY'] = groq_api_key
else:
    raise ValueError("GROQ_API_KEY is not set in the environment.")

PART 5 - MULTI QUERY

In [5]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

# Ensure the USER_AGENT environment variable is set before any HTTP requests
os.environ['USER_AGENT'] = 'your_custom_user_agent_string'  # Replace with your actual user agent

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split - Chunking
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Embed
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

# Initialize HuggingFaceEmbeddings (updated)
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Index with FAISS
vectorstore = FAISS.from_documents(documents=splits, embedding=hf_embeddings)

# Retrieve
retriever = vectorstore.as_retriever()


Multi-Query Prompt

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# Define the template for generating multiple perspectives on the user's question
template = """
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 

Provide these alternative questions separated by newlines.

Original question: {question}
"""

# Create the prompt template using ChatPromptTemplate
prompt_perspectives = ChatPromptTemplate.from_template(template)

# Define the pipeline for generating alternative questions
generate_queries = (
    prompt_perspectives  # Use the prompt template
    | ChatGroq(temperature=0)  # Use the ChatGroq model with zero temperature (deterministic responses)
    | StrOutputParser()  # Parse the output into a clean format (a single string with questions separated by newlines)
    | (lambda x: "\n".join([q.strip() for q in x.split("\n") if q.strip()]))  # Clean and join questions with newlines
)

# Example: Use the pipeline to generate alternative questions for a given user question
user_question = "What are the benefits of using LangChain for NLP tasks?"
alternative_questions = generate_queries.invoke({"question": user_question})

# Print the generated alternative questions
print(alternative_questions)

1. How does LangChain enhance natural language processing tasks?
2. Can you list the advantages of implementing LangChain in NLP projects?
3. In what ways does LangChain improve the efficiency of NLP tasks?
4. What are the key benefits of using LangChain for handling natural language processing tasks?
5. How can LangChain contribute to the success of NLP projects?


In [14]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

15

In [19]:
from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG template
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(temperature=0)

# Define the final RAG chain
final_rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")}  # Ensure the mapping is correct
    | prompt
    | llm
    | StrOutputParser()
    | (lambda x: "\n".join([q.strip() for q in x.split("\n") if q.strip()]))
)

# Define the question
question = "What is task decomposition for LLM agents?"

# Invoke the chain with the question and get the result
result = final_rag_chain.invoke({"question": question})

# Print the result
print(result)

Task decomposition for LLM agents refers to the process of breaking down a complex task into smaller, manageable subtasks that can be executed by a large language model (LLM) agent. This allows the agent to handle complex requests more effectively by addressing each subtask sequentially. The resulting subtasks can then be parsed, prioritized, and distributed to appropriate expert models for efficient processing. The order of tasks is important to consider due to logical relationships between them. If the user input cannot be parsed, the agent should return an empty JSON.
